In [1]:
import cv2
import numpy as np
import helpers
import sudoku

def showImage(img,title='Resized Window'):

    #define the screen resulation
    screen_res = 1280, 720
    scale_width = screen_res[0] / img.shape[1]
    scale_height = screen_res[1] / img.shape[0]
    scale = min(scale_width, scale_height)

    #resized window width and height
    window_width = int(img.shape[1] * scale)
    window_height = int(img.shape[0] * scale)

    #cv2.WINDOW_NORMAL makes the output window resizealbe
    cv2.namedWindow(title, cv2.WINDOW_NORMAL)

    #resize the window according to the screen resolution
    cv2.resizeWindow(title, window_width, window_height)

    cv2.imshow(title, img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [2]:
image = cv2.imread('puzz1.jpg',0)
outerBox = np.ndarray(image.shape)

In [3]:
image2 = sudoku.InputImage('test input', 'puzz1.jpg')
board2 = image2.getBoard()
board2.show()

In [4]:
board2.boxes[79].show()

<a href='https://www.tutorialkart.com/opencv/python/opencv-python-gaussian-image-smoothing/'>Gaussian Blur</a>

In [5]:
image = cv2.GaussianBlur(image,(11,11),0)

<a href="https://pyimagesearch.com/2021/05/12/adaptive-thresholding-with-opencv-cv2-adaptivethreshold/">Adaptive Thresholding</a>

In [6]:
outerBox = cv2.adaptiveThreshold(image,255,cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 5, 2)

In [7]:
cv2.adaptiveThreshold(image,255,cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 5, 2)
cv2.bitwise_not(outerBox)

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [8]:
outerBox = cv2.bitwise_not(outerBox)

<a href='https://docs.opencv.org/3.4/db/df6/tutorial_erosion_dilatation.html'>Dilation</a>

In [9]:
outerBox = cv2.dilate(outerBox, cv2.getStructuringElement(cv2.MORPH_CROSS,(3,3)),iterations=4)

In [10]:
contours, heirarchy = cv2.findContours(outerBox, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

In [11]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=4,random_state=0).fit(contours[0][:,0,:])

In [12]:
grid_corners = kmeans.cluster_centers_
image_corners_dict = {'topLeft':(0,0), 'topRight':(image.shape[1],0), 'bottomLeft':(0,image.shape[0]), 'bottomRight':(image.shape[1],image.shape[0])}
grid_corners_dict = {r: grid_corners[np.argmin([np.linalg.norm(i-np.array(k)) for i in grid_corners])] for (r,k) in image_corners_dict.items()}

In [13]:
list(image_corners_dict.values())

[(0, 0), (1125, 0), (0, 1167), (1125, 1167)]

In [14]:
x = int(min(image.shape[0],image.shape[1]) * 0.90)

pts1 = np.float32(list(grid_corners_dict.values()))
pts2 = np.float32([(0,0),(x,0),(0,x),(x,x)])
pts3 = np.float32(list(image_corners_dict.values())) * 0.90
#list(image_corners_dict.values())

In [15]:
matrix = cv2.getPerspectiveTransform(pts1, pts2)
newImage = cv2.warpPerspective(outerBox, matrix, image.shape)

In [16]:
showImage(newImage)

In [17]:
newImage = newImage[0:x,0:x]

In [18]:
boxes = board2.boxes.copy()

In [19]:
import os

from tensorflow import keras

#print(tf.version.VERSION)

In [20]:

new_model = sudoku.DigitRecognizer('digit_recognizer')
x = boxes[0].cv_image
showImage(new_model.preprocess(x))

In [47]:
model = keras.models.load_model('digit_recognizer')

In [51]:

test = boxes[76].cv_image
test = new_model.preprocess(test)
test = cv2.resize(test,(32,32),interpolation=cv2.INTER_AREA)
ret,test = cv2.threshold(test,240,255,cv2.THRESH_BINARY_INV)
showImage(test)
test = test.reshape((-1,32,32,1))

results = model.predict(test)
print(results)
print(np.argmax(results,axis = 1),int(np.max(results)))


[[0.000000e+00 0.000000e+00 0.000000e+00 1.000000e+00 2.297281e-37
  0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00 7.070188e-37]]
[3] 1


In [40]:
test = boxes[4]
test = cv2.resize(test,(32,32),interpolation=cv2.INTER_AREA)

showImage(test)
test = test.reshape((-1,32,32,1))
results = new_model.model.predict(test)
print(results)
print(np.argmax(results,axis = 1),int(np.max(results)))

error: OpenCV(4.5.5) :-1: error: (-5:Bad argument) in function 'resize'
> Overload resolution failed:
>  - src is not a numpy array, neither a scalar
>  - Expected Ptr<cv::UMat> for argument 'src'


In [ ]:
test= cv2.dilate(test, cv2.getStructuringElement(cv2.MORPH_CROSS,(3,3)),iterations=3)
test = cv2.bilateralFilter(test,4,75,75)
test = cv2.resize(test,(32,32),interpolation=cv2.INTER_AREA)
test = cv2.bitwise_not(test)

In [84]:
for test in boxes:
    test = cv2.resize(test,(32,32),interpolation=cv2.INTER_AREA)
    test = test.reshape((-1,32,32,1))
    results = new_model.predict(test)
    print(np.argmax(results,axis = 1),np.max(results))


[9] 1.0
[2] 1.0
[2] 0.79571104
[2] 0.7891244
[3] 0.9918208
[2] 0.94732153
[3] 1.0
[2] 0.9995128
[2] 0.9989851
[8] 1.0
[5] 1.0
[8] 0.89203185
[8] 0.73443794
[9] 1.0
[8] 0.9383874
[8] 0.97968984
[2] 1.0
[8] 0.9502507
[6] 0.99999535
[8] 0.7601064
[3] 1.0
[8] 0.9967078
[8] 0.9999789
[8] 0.9999317
[8] 0.99991786
[8] 0.99926275
[8] 0.999741
[6] 0.99999964
[6] 0.98486876
[8] 0.9728976
[6] 0.58393675
[8] 0.9990087
[2] 1.0
[8] 0.9996209
[8] 0.9973156
[8] 0.99878615
[3] 1.0
[8] 0.9683564
[8] 0.99994993
[8] 0.99858284
[8] 0.9999862
[8] 0.96292126
[6] 1.0
[8] 0.9997408
[8] 0.99993813
[2] 1.0
[9] 1.0
[2] 1.0
[8] 0.999997
[8] 0.9999999
[8] 0.99999607
[2] 1.0
[5] 1.0
[8] 0.9991406
[6] 0.9174445
[8] 0.9999579
[8] 0.9999999
[5] 1.0
[8] 1.0
[9] 1.0
[8] 0.99999964
[6] 0.9998722
[2] 1.0
[8] 0.9810554
[8] 1.0
[5] 1.0
[8] 0.9999999
[2] 1.0
[8] 1.0
[4] 1.0
[8] 0.9999995
[8] 0.99999714
[8] 0.9999939
[8] 1.0
[9] 0.99999166
[2] 1.0
[4] 1.0
[8] 1.0
[8] 1.0
[3] 1.0
[8] 0.9999987


In [26]:
test = test.reshape((-1,32,32,1))
results = new_model.predict(test)
results = np.argmax(results,axis = 1)
#np.max(results)

In [47]:
showImage(outerBox[50:850,50:850])

In [48]:
showImage(image)

In [28]:
results

array([8])

In [49]:

cirle = image.copy()
# Radius of circle
radius = 2
  
# Blue color in BGR
color = (255, 0, 0)
  
# Line thickness of 2 px
thickness = 2
  
# Using cv2.circle() method
# Draw a circle with blue line borders of thickness of 2 px
for c  in np.int32(list(grid_corners)):
    circle = cv2.circle(image,c, radius, color, thickness)

In [50]:
showImage(circle)